# Data Engineering Capstone Project Happy

#### Project Summary
The goal of this project was to create ETL pipleine for immigration, airport data, state and world happiness rating data. We could try to find answears to questions such as, do people from countries with unhappy or happy rating immigrate to the US in large numbers, or which airport is more frequent for immigrants.


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Import libraries

In [1]:
# Imports and installs
import pandas as pd
import importlib
import os
import configparser
import datetime as dt

from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import isnan, when, count, col, udf, dayofmonth, dayofweek, month, year, weekofyear



### Load Configuration Data and Spark Session

In [2]:
config = configparser.ConfigParser()
config.read('iam.cfg')
os.environ['AWS_ACCESS_KEY_ID']=config['AWS_CREDS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS_CREDS']['AWS_SECRET_ACCESS_KEY']

spark = SparkSession.builder.\
    config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11").\
    enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

To create the database through the following steps:
* Use Spark to load the data into dataframes.
* Data analysis and data cleaning of I94 immigration dataset, airport dataset, state and world happiness rating data.
* Create immigration calendar dimension table from I94 immigration dataset and this table links to the fact table through the arrdate field.
* Create country dimension table from the I94 immigration dataset. 
* Create airport dimension table from the airport data and this table links to the fact table through the i94port field.
* Create happiness dimension table from the happiness data and this table links to the fact table through the state code field.
* Create visa dimension table from the I94 immigration data and this table links to the fact table through the visa_type_key field.
* Create fact table from the clean I94 immigration dataset and the visa_type dimension.

#### I94 Immigration dataset
<hr style="background-color: #b7d0e2;"/> 

Import and check immigration dataset

In [3]:
# Read in the data and count
imm_df =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
imm_df.count()

3096313

In [4]:
# display the first records
imm_df.limit(5).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,None,NaN,...,U,None,1979.0,10282016,None,None,None,1.897628e+09,None,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,1988.0,09302016,None,None,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,2012.0,09302016,None,None,AA,9.246846e+10,00199,B2


In [9]:
# dataframe schema
imm_df.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

<b><i>Data dictionary - I94 Immigration dataset</i></b>

<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">cicid</td><td class="tg-0pky">Unique record ID</td>
 <tr><td class="tg-0pky">i94yr</td><td class="tg-0pky">xxxx digit year</td>
 <tr><td class="tg-0pky">i94mon</td><td class="tg-0pky">xx numeric month</td>
 <tr><td class="tg-0pky">i94cit</td><td class="tg-0pky">xxx code for immigrant country of birth</td>
 <tr><td class="tg-0pky">i94res</td><td class="tg-0pky">xxx code for immigrant country of residence </td>
 <tr><td class="tg-0pky">i94port</td><td class="tg-0pky">xxx code Port of admission</td>
 <tr><td class="tg-0pky">arrdate</td><td class="tg-0pky">Arrival Date in the USA</td>
 <tr><td class="tg-0pky">i94mode</td><td class="tg-0pky">Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported)</td>
 <tr><td class="tg-0pky">i94addr</td><td class="tg-0pky">USA State of arrival</td>
 <tr><td class="tg-0pky">depdate</td><td class="tg-0pky">Departure Date from the USA</td>
 <tr><td class="tg-0pky">i94bir</td><td class="tg-0pky">Age of Respondent in Years</td>
 <tr><td class="tg-0pky">i94visa</td><td class="tg-0pky">Visa codes categories - 1 = Business; 2 = Pleasure; 3 = Student</td>
 <tr><td class="tg-0pky">count</td><td class="tg-0pky">Field used for summary statistics</td>
 <tr><td class="tg-0pky">dtadfile</td><td class="tg-0pky">Character Date Field - Date added to I-94 Files</td>
 <tr><td class="tg-0pky">visapost</td><td class="tg-0pky">Department of State where where Visa was issued</td>
 <tr><td class="tg-0pky">occup</td><td class="tg-0pky">Occupation that will be performed in U.S - CIC does not use</td>
 <tr><td class="tg-0pky">entdepa</td><td class="tg-0pky">Arrival Flag - admitted or paroled into the U.S.</td>
 <tr><td class="tg-0pky">entdepd</td><td class="tg-0pky">Departure Flag - Departed, lost I-94 or is deceased</td>
 <tr><td class="tg-0pky">entdepu</td><td class="tg-0pky">Update Flag - Either apprehended, overstayed, adjusted to perm residence - CIC does not use</td>
 <tr><td class="tg-0pky">matflag</td><td class="tg-0pky">Match flag - Match of arrival and departure records</td>
 <tr><td class="tg-0pky">biryear</td><td class="tg-0pky">4 digit year of birth</td>
 <tr><td class="tg-0pky">dtaddto</td><td class="tg-0pky">Character Date Field - Date to which admitted to U.S. (allowed to stay until)</td>
 <tr><td class="tg-0pky">gender</td><td class="tg-0pky">Non-immigrant sex</td>
 <tr><td class="tg-0pky">insnum</td><td class="tg-0pky">INS number</td>
 <tr><td class="tg-0pky">airline</td><td class="tg-0pky">Airline used to arrive in U.S.</td>
 <tr><td class="tg-0pky">admnum</td><td class="tg-0pky">Admission Number</td>
 <tr><td class="tg-0pky">fltno</td><td class="tg-0pky">Flight number of Airline used to arrive in U.S.</td>
 <tr><td class="tg-0pky">visatype</td><td class="tg-0pky">Class of admission legally admitting the non-immigrant to temporarily stay in U.S.</td>
</table>

#### Airport dataset
<hr style="background-color: #b7d0e2;"/> 

Import and check Airport dataset - iata code is important

In [6]:
# Read in the data and count
file_name = "airport-codes_csv.csv"
airport_df = spark.read.csv(file_name, inferSchema=True, header=True, sep=',')
airport_df.count()

55075

In [7]:
# display the first records
airport_df.limit(5).toPandas()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"


In [8]:
# dataframe schema
airport_df.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



<b><i>Data dictionary - Airport dataset</i></b>

<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">ident</td><td class="tg-0pky">Ident</td>
 <tr><td class="tg-0pky">type</td><td class="tg-0pky">Type of airport</td>
 <tr><td class="tg-0pky">name</td><td class="tg-0pky">Name of airport</td>
 <tr><td class="tg-0pky">elevation_ft</td><td class="tg-0pky">Elevation of airport in ft</td>
 <tr><td class="tg-0pky">continent</td><td class="tg-0pky">Continent</td>
 <tr><td class="tg-0pky">iso_country</td><td class="tg-0pky">Iso country code</td>     
 <tr><td class="tg-0pky">iso_region</td><td class="tg-0pky">Iso region code</td>
 <tr><td class="tg-0pky">municipality</td><td class="tg-0pky">Municipality of airport</td>     
 <tr><td class="tg-0pky">gps_code</td><td class="tg-0pky">Gps code of airport</td>     
 <tr><td class="tg-0pky">iata_code</td><td class="tg-0pky">Iata code of airport</td>     
 <tr><td class="tg-0pky">local_code</td><td class="tg-0pky">Local code of airport</td>     
 <tr><td class="tg-0pky">coordinates</td><td class="tg-0pky">Coordinates od airport</td>         
</table>

#### World Happiness dataset
<hr style="background-color: #b7d0e2;"/> 

Import and check world happiness dataset for 2016 year

In [10]:
# Read in the data and count
file_name = "happiness_2016.csv"
happy_df = spark.read.csv(file_name, inferSchema=True, header=True, sep=',')
happy_df.count()

157

In [11]:
# display the first records
happy_df.limit(5).toPandas()

,Country,Region,Happy_Rank,Happy_Score,lconf_level,Hconf_level,Economy,Family,Health,Freedom,Trust,Generosity,Dystopia_res
0,Denmark,Western Europe,1,7.526,7.460,7.592,1.44178,1.16374,0.79504,0.57941,0.44453,0.36171,2.73939
1,Switzerland,Western Europe,2,7.509,7.428,7.590,1.52733,1.14524,0.86303,0.58557,0.41203,0.28083,2.69463
2,Iceland,Western Europe,3,7.501,7.333,7.669,1.42666,1.18326,0.86733,0.56624,0.14975,0.47678,2.83137
3,Norway,Western Europe,4,7.498,7.421,7.575,1.57744,1.12690,0.79579,0.59609,0.35776,0.37895,2.66465
4,Finland,Western Europe,5,7.413,7.351,7.475,1.40598,1.13464,0.81091,0.57104,0.41004,0.25492,2.82596


In [12]:
# dataframe schema
happy_df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Happy_Rank: integer (nullable = true)
 |-- Happy_Score: double (nullable = true)
 |-- lconf_level: double (nullable = true)
 |-- Hconf_level: double (nullable = true)
 |-- Economy: double (nullable = true)
 |-- Family: double (nullable = true)
 |-- Health: double (nullable = true)
 |-- Freedom: double (nullable = true)
 |-- Trust: double (nullable = true)
 |-- Generosity: double (nullable = true)
 |-- Dystopia_res: double (nullable = true)



<b><i>Data dictionary -  World Happiness dataset</i></b>

<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">Country</td><td class="tg-0pky">Name of country</td>
 <tr><td class="tg-0pky">Region</td><td class="tg-0pky">Region the country</td>
 <tr><td class="tg-0pky">Happiness_rank</td><td class="tg-0pky">Rank of the country based on the Happiness Score</td>
 <tr><td class="tg-0pky">Happiness_score</td><td class="tg-0pky">A metric measured in 2015 by asking the sampled people the question: "How would you rate your happiness"</td>
 <tr><td class="tg-0pky">Lower_confidence_interval</td><td class="tg-0pky">Lower Confidence Interval of the Happiness Score</td>
 <tr><td class="tg-0pky">Upper_confidence_interval</td><td class="tg-0pky">Upper Confidence Interval of the Happiness Score</td>     
 <tr><td class="tg-0pky">Economy</td><td class="tg-0pky">The extent to which GDP contributes to the calculation of the Happiness Score.</td>
 <tr><td class="tg-0pky">Family</td><td class="tg-0pky">The extent to which Family contributes to the calculation of the Happiness Score</td>     
 <tr><td class="tg-0pky">Health</td><td class="tg-0pky">The extent to which Life expectancy contributed to the calculation of the Happiness Score</td>     
 <tr><td class="tg-0pky">Freedom</td><td class="tg-0pky">The extent to which Freedom contributed to the calculation of the Happiness Score</td>     
 <tr><td class="tg-0pky">Trust</td><td class="tg-0pky">The extent to which Perception of Corruption contributes to Happiness Score</td>     
 <tr><td class="tg-0pky">Generosity</td><td class="tg-0pky">The extent to which Generosity contributed to the calculation of the Happiness Score</td>         
 <tr><td class="tg-0pky">Dystopia_residual</td><td class="tg-0pky">The extent to which Dystopia Residual contributed to the calculation of the Happiness Score.</td>         
</table>

### Step 2: Explore and Assess the Data
#### Explore, cleaning and data analysis

#### Cleaning Steps - Immigration data

In [13]:
# Performing cleaning tasks
imm_df.count()

# create a nan dataframe 
nan_df = imm_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in imm_df.columns]).toPandas()

# display nan sum
nan_df.sum()

cicid             0
i94yr             0
i94mon            0
i94cit            0
i94res            0
i94port           0
arrdate           0
i94mode         239
i94addr      152592
depdate      142457
i94bir          802
i94visa           0
count             0
dtadfile          1
visapost    1881250
occup       3088187
entdepa         238
entdepd      138429
entdepu     3095921
matflag      138429
biryear         802
dtaddto         477
gender       414269
insnum      2982605
airline       83627
admnum            0
fltno         19549
visatype          0
dtype: int64

In [14]:
# columns for drop - over 2.000.000 nan values
cols = ['occup', 'entdepu','insnum']

# drop columns
new_imm_df = imm_df.drop(*cols)

In [15]:
# Total records after dropping rows with missing values

new_imm_df.count()

3096313

#### Cleaning Steps - Airport data

In [16]:
# Performing cleaning tasks
airport_df.count()

# create a nan dataframe 
nan_df = airport_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in airport_df.columns]).toPandas()

# display nan sum
nan_df.sum()

ident               0
type                0
name                0
elevation_ft     7006
continent           0
iso_country         0
iso_region          0
municipality     5676
gps_code        14045
iata_code       45886
local_code      26389
coordinates         0
dtype: int64

In [17]:
# drop rows with missing iata_code
airport_df = airport_df.dropna(subset=['iata_code'])

In [18]:
# Total records after dropping rows with missing values

airport_df.count()

9189

### Step 3: Define the Data Model

### 3.1 Conceptual Data Model
![Database schema](Capstone_project_happy_data_model.jpg)

In this project we have 5 dimensions table and 1 fact table. 
The airport dimension table comes from airport_codes_csv and link to fact table through the i94port field.
The country dimension table is made up of data from the immigration datasets and link to fact table through the country_code field.
The happiness dimension table comes from the happiness dataset and links to the immigration fact table through the country_code field. 
The visa type dimension table comes from the immigration datasets and links to the immigaration through the visa_type_key field. 
The immigration fact table is the heart of the data model. This table's data comes from the immigration data sets and contains keys that links to the dimension tables.

### 3.2 Mapping Out Data Pipelines
The pipeline steps are as follows:
* Load the datasets and analysis data
* Clean all data missing data and column 
* Create visa_type, country, immigration calendar, happiness and airport dimension table
* Create immigration fact table
* Checking the number of data

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

##### Create immcalendar dimension table
---

In [21]:
# Create immcalendar dimension table with new_imm_df

# definition output dir and create data
output_data = "data_tables/"

# create udf to convert arrival date to datetime object
get_datetime = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(x)).isoformat() if x else None)

immcal_df = new_imm_df

# create calendar df from arrdate column
immcal_df = new_imm_df.select(['arrdate']).withColumn("arrdate", get_datetime(immcal_df.arrdate)).distinct()
    
# expand df adding other columns
immcal_df = immcal_df.withColumn('arrival_day', dayofmonth('arrdate'))
immcal_df = immcal_df.withColumn('arrival_week', weekofyear('arrdate'))
immcal_df = immcal_df.withColumn('arrival_month', month('arrdate'))
immcal_df = immcal_df.withColumn('arrival_year', year('arrdate'))
immcal_df = immcal_df.withColumn('arrival_weekday', dayofweek('arrdate'))

# create id 
immcal_df = immcal_df.withColumn('id', monotonically_increasing_id())
    
# write to parquet file
immcal_df.write.parquet(output_data + "immigration_calendar", mode="overwrite")

In [22]:
# show first 5 
immcal_df.show(5)

+----------+-----------+------------+-------------+------------+---------------+------------+
|   arrdate|arrival_day|arrival_week|arrival_month|arrival_year|arrival_weekday|          id|
+----------+-----------+------------+-------------+------------+---------------+------------+
|2026-04-15|         15|          16|            4|        2026|              4| 34359738368|
|2026-04-07|          7|          15|            4|        2026|              3| 68719476736|
|2026-04-24|         24|          17|            4|        2026|              6|154618822656|
|2026-05-01|          1|          18|            5|        2026|              6|180388626432|
|2026-04-10|         10|          15|            4|        2026|              6|249108103168|
+----------+-----------+------------+-------------+------------+---------------+------------+
only showing top 5 rows



##### Create the country dimension table
---

In [30]:
# Create the country dimension table with new_imm_df

# load state csv
map_codes = pd.read_csv('i94_state.csv')

@udf()
def get_country_name(code):
    name = map_codes[map_codes['code']==code]['Name'].iloc[0]
    if name:
        return name.title()
    return None
    
# select and rename i94_state column
country_df = new_imm_df.select(['i94res']).distinct().withColumnRenamed('i94res', 'country_code')
    
# create country_name column
country_df = country_df.withColumn('country_name', get_country_name(country_df.country_code))
   
# write the dimension to a parquet file
country_df.write.parquet(output_data + "country", mode="overwrite")

In [31]:
# show first 5 
country_df.show(5)

+------------+------------+
|country_code|country_name|
+------------+------------+
|       692.0|     Ecuador|
|       299.0|    Mongolia|
|       576.0| El Salvador|
|       735.0|  Montenegro|
|       206.0|   Hong Kong|
+------------+------------+
only showing top 5 rows



##### Create visa dimension table
---

In [32]:
# Create visa dimension table with new_imm_df

# create visa df 
visa_df = new_imm_df.select(['visatype']).distinct()
   
# add id column
visa_df = visa_df.withColumn('visa_key', monotonically_increasing_id())
    
# write dimension to parquet file
visa_df.write.parquet(output_data + "visatype", mode="overwrite")

In [34]:
# show first 5 
visa_df.show(5)

+--------+------------+
|visatype|    visa_key|
+--------+------------+
|      F2|103079215104|
|     GMB|352187318272|
|      B2|369367187456|
|      F1|498216206336|
|     CPL|601295421440|
+--------+------------+
only showing top 5 rows



##### Create airport dimension table
---

In [37]:
# Create airport dimension table
     
# write dimension to parquet file
airport_df.write.parquet(output_data + "airport", mode="overwrite")

In [38]:
# show first 5 
airport_df.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+-------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region| municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+-------------+--------+---------+----------+--------------------+
|  03N|small_airport|      Utirik Airport|           4|       OC|         MH|    MH-UTI|Utirik Island|    K03N|      UTK|       03N|  169.852005, 11.222|
| 07FA|small_airport|Ocean Reef Club A...|           8|       NA|         US|     US-FL|    Key Largo|    07FA|      OCA|      07FA|-80.274803161621,...|
|  0AK|small_airport|Pilot Station Air...|         305|       NA|         US|     US-AK|Pilot Station|    null|      PQS|       0AK|-162.899994, 61.9...|
| 0CO2|small_airport|Crested Butte Air...|        8980|       NA|         US

##### Create happiness fact table
---

In [39]:
# Create happiness fact table
    
# load state csv
map_codes = pd.read_csv('i94_state.csv')
    
# Title name
map_codes['Name'] = map_codes['Name'].str.title() 
    
@udf()
def get_country_code(name):
    code = map_codes[map_codes['Name'] == name]['code']
    if not code.empty:
         return str(code.iloc[0])
    else:
         return None

# create country_id column
happy_df = happy_df.withColumn('Country_id', get_country_code(happy_df.Country))
    
# write dimension to parquet file
happy_df.write.parquet(output_data + "happiness", mode="overwrite")

In [40]:
# show first 5 
happy_df.show(5)

+-----------+--------------+----------+-----------+-----------+-----------+-------+-------+-------+-------+-------+----------+------------+----------+
|    Country|        Region|Happy_Rank|Happy_Score|lconf_level|Hconf_level|Economy| Family| Health|Freedom|  Trust|Generosity|Dystopia_res|Country_id|
+-----------+--------------+----------+-----------+-----------+-----------+-------+-------+-------+-------+-------+----------+------------+----------+
|    Denmark|Western Europe|         1|      7.526|       7.46|      7.592|1.44178|1.16374|0.79504|0.57941|0.44453|   0.36171|     2.73939|       108|
|Switzerland|Western Europe|         2|      7.509|      7.428|       7.59|1.52733|1.14524|0.86303|0.58557|0.41203|   0.28083|     2.69463|       131|
|    Iceland|Western Europe|         3|      7.501|      7.333|      7.669|1.42666|1.18326|0.86733|0.56624|0.14975|   0.47678|     2.83137|       115|
|     Norway|Western Europe|         4|      7.498|      7.421|      7.575|1.57744| 1.1269|0.7

##### Create immigration fact table
---

In [43]:
# Create immigration fact table
   
# read visa_dim
dim_df = spark.read.parquet(output_data + "visatype").toPandas()
        
@udf('string')
def get_visa_key(visa_type):
    key_series = dim_df[dim_df['visatype']==visa_type]['visa_key']
        
    if not key_series.empty:
        return str(key_series.iloc[0])
    return None
       
# create udf to convert arrival date to datetime object
get_datetime = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(x)).isoformat() if x else None)
    
# rename columns 
new_imm_df = new_imm_df.withColumnRenamed('cicid','id') \
            .withColumnRenamed('i94res', 'country_code') \
            .withColumnRenamed('i94addr', 'state_code') 
    
# create visa_type key
new_imm_df = new_imm_df.withColumn('visa_type_key', get_visa_key('visatype'))
    
# convert arrival date into datetime object
new_imm_df = new_imm_df.withColumn("arrdate", get_datetime(new_imm_df.arrdate))
    
# write dimension to parquet file
new_imm_df.write.parquet(output_data + "immigration_fact", mode="overwrite")


In [45]:
new_imm_df.show(5)

+----+------+------+------+------------+-------+----------+-------+----------+-------+------+-------+-----+--------+--------+-------+-------+-------+-------+--------+------+-------+--------------+-----+--------+-------------+
|  id| i94yr|i94mon|i94cit|country_code|i94port|   arrdate|i94mode|state_code|depdate|i94bir|i94visa|count|dtadfile|visapost|entdepa|entdepd|matflag|biryear| dtaddto|gender|airline|        admnum|fltno|visatype|visa_type_key|
+----+------+------+------+------------+-------+----------+-------+----------+-------+------+-------+-----+--------+--------+-------+-------+-------+-------+--------+------+-------+--------------+-----+--------+-------------+
| 6.0|2016.0|   4.0| 692.0|       692.0|    XXX|2026-04-30|   null|      null|   null|  37.0|    2.0|  1.0|    null|    null|      T|   null|   null| 1979.0|10282016|  null|   null| 1.897628485E9| null|      B2| 369367187456|
| 7.0|2016.0|   4.0| 254.0|       276.0|    ATL|2026-04-08|    1.0|        AL|   null|  25.0|   

#### 4.2 Data Quality Checks
Data count checks on fact and dimension table to be ensure that ETL procedure works fine.

In [49]:
# Perform quality checks here    

total_count = new_imm_df.count()
print(f"Data quality check for immigration_fact with {total_count} records")   

total_count = visa_df.count()
print(f"Data quality check for visa_dim with {total_count} records")  

total_count = airport_df.count()
print(f"Data quality check for airport_dim with {total_count} records")  

total_count = country_df.count()
print(f"Data quality check for country_dim with {total_count} records")  

total_count = immcal_df.count()
print(f"Data quality check for imcalendar_dim with {total_count} records")  

total_count = happy_df.count()
print(f"Data quality check for happiness_dim with {total_count} records")  

Data quality check for immigration_fact with 3096313 records
Data quality check for visa_dim with 17 records
Data quality check for airport_dim with 9189 records
Data quality check for country_dim with 229 records
Data quality check for imcalendar_dim with 30 records
Data quality check for happiness_dim with 157 records


#### 4.3 Data dictionary 

##### Fact Table - data dictionary

<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">id</td><td class="tg-0pky">Unique record ID</td></tr>
 <tr><td class="tg-0pky">country_code</td><td class="tg-0pky">3 digit code for immigrant country of residence </td></tr>    
 <tr><td class="tg-0pky">visa_type_key</td><td class="tg-0pky">A numerical key that links to the visa_type dimension table</td></tr>
 <tr><td class="tg-0pky">state_code</td><td class="tg-0pky">US state of arrival</td></tr>
 <tr><td class="tg-0pky">i94cit</td><td class="tg-0pky">country code for immigrant country of birth</td></tr>
 <tr><td class="tg-0pky">i94yr</td><td class="tg-0pky">4 digit year</td></tr>
 <tr><td class="tg-0pky">i94mon</td><td class="tg-0pky">Numeric month</td></tr>
 <tr><td class="tg-0pky">i94port</td><td class="tg-0pky">Port of admission</td></tr>
 <tr><td class="tg-0pky">arrdate</td><td class="tg-0pky">Arrival Date in the USA</td></tr>
 <tr><td class="tg-0pky">i94mode</td><td class="tg-0pky">Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported)</td></tr>
 <tr><td class="tg-0pky">depdate</td><td class="tg-0pky">Departure Date from the USA</td></tr>
 <tr><td class="tg-0pky">i94bir</td><td class="tg-0pky">Age of Respondent in Years</td></tr>
 <tr><td class="tg-0pky">i94visa</td><td class="tg-0pky">Visa codes collapsed into three categories</td></tr>
 <tr><td class="tg-0pky">count</td><td class="tg-0pky">Field used for summary statistics</td></tr>
 <tr><td class="tg-0pky">dtadfile</td><td class="tg-0pky">Character Date Field - Date added to I-94 Files</td></tr>
 <tr><td class="tg-0pky">visapost</td><td class="tg-0pky">Department of State where where Visa was issued </td></tr>
 <tr><td class="tg-0pky">entdepa</td><td class="tg-0pky">Arrival Flag - admitted or paroled into the U.S.</td></tr>
 <tr><td class="tg-0pky">entdepd</td><td class="tg-0pky">Departure Flag - Departed, lost I-94 or is deceased</td></tr>
 <tr><td class="tg-0pky">matflag</td><td class="tg-0pky">Match flag - Match of arrival and departure records</td></tr>
 <tr><td class="tg-0pky">biryear</td><td class="tg-0pky">4 digit year of birth</td></tr>
 <tr><td class="tg-0pky">dtaddto</td><td class="tg-0pky">Character Date Field - Date to which admitted to U.S. (allowed to stay until)</td></tr>
 <tr><td class="tg-0pky">gender</td><td class="tg-0pky">Non-immigrant sex</td></tr>
 <tr><td class="tg-0pky">airline</td><td class="tg-0pky">Airline used to arrive in U.S.</td>
 <tr><td class="tg-0pky">admnum</td><td class="tg-0pky">Admission Number</td>
 <tr><td class="tg-0pky">fltno</td><td class="tg-0pky">Flight number of Airline used to arrive in U.S.</td>
</table>

##### Country Dimension Table - data dictionary
<p>  
<i>The country code and country_name fields</i>
</p>
<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">country_code</td><td class="tg-0pky">Unique country code</td></tr>
 <tr><td class="tg-0pky">country_name</td><td class="tg-0pky">Name of country</td></tr>    
 </table>

##### Visa Type Dimension Table - data dictionary
<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">visa_key</td><td class="tg-0pky">Unique id for each visa issued</td></tr>
 <tr><td class="tg-0pky">visa_type</td><td class="tg-0pky">Name of visa</td></tr>
</table>

 ##### Immigration Calendar Dimension Table - data dictionary
<p>
<i>The whole of this dataset comes from the immigration dataset.</i>
</p>
<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">id</td><td class="tg-0pky">Unique id</td></tr>
 <tr><td class="tg-0pky">arrdate</td><td class="tg-0pky">Arrival date into US</td></tr>    
 <tr><td class="tg-0pky">arrival_year</td><td class="tg-0pky">Arrival year into US</td></tr>
 <tr><td class="tg-0pky">arrival_month</td><td class="tg-0pky">Arrival MonthS</td></tr>
 <tr><td class="tg-0pky">arrival_day</td><td class="tg-0pky">Arrival Day</td></tr>
 <tr><td class="tg-0pky">arrival_week</td><td class="tg-0pky">Arrival Week</td></tr>
 <tr><td class="tg-0pky">arrival_weekday</td><td class="tg-0pky">Arrival WeekDay</td></tr>
</table>

##### Airport Dimension Table - data dictionary
<p>  
<i>The airport data comes from csv file.</i>
</p>
<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">ident</td><td class="tg-0pky">Identity</td></tr>
 <tr><td class="tg-0pky">type</td><td class="tg-0pky">Type</td></tr>    
 <tr><td class="tg-0pky">name</td><td class="tg-0pky">Name</td></tr>
 <tr><td class="tg-0pky">elevation_ft</td><td class="tg-0pky">Elevation_ft</td></tr>
 <tr><td class="tg-0pky">continent</td><td class="tg-0pky">Continent</td></tr>
 <tr><td class="tg-0pky">iso_county</td><td class="tg-0pky">Iso county</td></tr>
 <tr><td class="tg-0pky">iso_region</td><td class="tg-0pky">Iso region</td></tr>
 <tr><td class="tg-0pky">municipality</td><td class="tg-0pky">Municipality</td></tr>
 <tr><td class="tg-0pky">gps_code</td><td class="tg-0pky">Gps code</td></tr>
 <tr><td class="tg-0pky">iata_code</td><td class="tg-0pky">Iata code</td></tr>
 <tr><td class="tg-0pky">coordinates</td><td class="tg-0pky">Coordinates</td></tr>
    
</table>

##### Happiness Dimension Table - data dictionary
<p>
<i>The whole of this dataset comes from World Happiness Report 2016.</i>
</p>

<table class="tg" align="left">
  <tr>
    <th class="tg-0pky">Feature</th>
    <th class="tg-0pky">Description</th>
  </tr>
 <tr><td class="tg-0pky">country_id</td><td class="tg-0pky">Country id</td>
 <tr><td class="tg-0pky">country</td><td class="tg-0pky">Name of the country.</td>
 <tr><td class="tg-0pky">region</td><td class="tg-0pky">Region the country belongs to.</td>
 <tr><td class="tg-0pky">happy_rank</td><td class="tg-0pky">Rank of the country based on the Happiness Score</td>
 <tr><td class="tg-0pky">happy_score</td><td class="tg-0pky">A metric measured by asking the sampled people the question: "How would you rate your happiness"</td>
 <tr><td class="tg-0pky">lconf_level</td><td class="tg-0pky">Lower Confidence Interval of the Happiness Score</td>
 <tr><td class="tg-0pky">hconf_level</td><td class="tg-0pky">Upper Confidence Interval of the Happiness Score</td>
 <tr><td class="tg-0pky">economy</td><td class="tg-0pky">The extent to which GDP contributes to the calculation of the Happiness Score.</td>
 <tr><td class="tg-0pky">family</td><td class="tg-0pky">The extent to which Family contributes to the calculation of the Happiness Score</td>
 <tr><td class="tg-0pky">healty</td><td class="tg-0pky">The extent to which Life expectancy contributed to the calculation of the Happiness Score</td>
 <tr><td class="tg-0pky">freedom</td><td class="tg-0pky">The extent to which Freedom contributed to the calculation of the Happiness Score</td>
 <tr><td class="tg-0pky">trust</td><td class="tg-0pky">The extent to which Perception of Corruption contributes to Happiness Score</td>
 <tr><td class="tg-0pky">generosity</td><td class="tg-0pky">The extent to which Generosity contributed to the calculation of the Happiness Score</td>
 <tr><td class="tg-0pky">dystopia_res</td><td class="tg-0pky">The extent to which Dystopia Residual contributed to the calculation of the Happiness Score.</td>
</table>

#### Step 5: Complete Project Write Up
* I use Apache Spark beacause it's ability to handle multiple file formats with large amounts of data and it's fast for big data and large dataset. Immigration data will be updated mounthly and other data yearly. 
 * The data was increased by 100x -  Spark can easyly increase number of nodes in our AWS cluster.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day - We can use Apache Airflow and schedule data pipline.
 * The database needed to be accessed by 100+ people - We can use Amazon Redshift.